In [1]:
# Basic Libraries
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from wordcloud import WordCloud
import re

In [2]:
##natural language processing
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

In [3]:
##for machine learning
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier

In [4]:
#importing
reviewcleansed_dataframe= pd.read_csv('6.review(cleansed).csv')

In [5]:
import numpy as np
current_state = np.random.get_state()
a = np.random.randn(3)

In [6]:
reviewcleansed_dataframe.head()

,Unnamed: 0,listing_id,id,date,reviewer_id,reviewer_name,comments,comments_nostop,comments_tokenized,stemmed,lemmatized,compound,neg,neu,pos,bin,bin_word
0,0,50646.0,11909864.0,18/4/2014,1356099.0,James,A COMFORTABLE ROOM IN A SMART CONDO DEVELOPMEN...,A COMFORTABLE ROOM SMART CONDO DEVELOPMENT EVE...,"['A', 'COMFORTABLE', 'ROOM', 'SMART', 'CONDO',...","['A', 'COMFORT', 'ROOM', 'SMART', 'CONDO', 'DE...","['A', 'COMFORTABLE', 'ROOM', 'SMART', 'CONDO',...",0.9298,0.054,0.623,0.323,6.0,VP
1,1,50646.0,13823948.0,5/6/2014,15222393.0,Welli,STAYED OVER AT SUJATHAS HOUSE FOR GOOD NIGHTS...,STAYED SUJATHAS HOUSE GOOD NIGHTS BOYFRIEND SU...,"['STAYED', 'SUJATHAS', 'HOUSE', 'GOOD', 'NIGHT...","['STAY', 'SUJATHA', 'HOUS', 'GOOD', 'NIGHT', '...","['STAYED', 'SUJATHAS', 'HOUSE', 'GOOD', 'NIGHT...",0.9878,0.000,0.613,0.387,6.0,VP
2,2,50646.0,15117222.0,2/7/2014,5543172.0,Cyril,ITS BEEN A LOVELY STAY AT SUJATHAS THE ROOM IS...,ITS LOVELY STAY SUJATHAS THE ROOM CLEAN LOCATI...,"['ITS', 'LOVELY', 'STAY', 'SUJATHAS', 'THE', '...","['IT', 'LOVE', 'STAY', 'SUJATHA', 'THE', 'ROOM...","['ITS', 'LOVELY', 'STAY', 'SUJATHAS', 'THE', '...",0.9627,0.000,0.645,0.355,6.0,VP
3,3,50646.0,15426462.0,8/7/2014,817532.0,Jake,WE HAD A GREAT EXPERIENCE A NICE PLACE AN AMAZ...,WE GREAT EXPERIENCE A NICE PLACE AMAZING COMPL...,"['WE', 'GREAT', 'EXPERIENCE', 'A', 'NICE', 'PL...","['WE', 'GREAT', 'EXPERI', 'A', 'NICE', 'PLACE'...","['WE', 'GREAT', 'EXPERIENCE', 'A', 'NICE', 'PL...",0.9274,0.000,0.489,0.511,6.0,VP
4,4,50646.0,15552912.0,11/7/2014,10942382.0,Subba,QUIET CONDO COMFORTABLE STAY AND GOOD LOCATION,QUIET CONDO COMFORTABLE STAY GOOD LOCATION,"['QUIET', 'CONDO', 'COMFORTABLE', 'STAY', 'GOO...","['QUIET', 'CONDO', 'COMFORT', 'STAY', 'GOOD', ...","['QUIET', 'CONDO', 'COMFORTABLE', 'STAY', 'GOO...",0.7351,0.000,0.446,0.554,6.0,VP


In [7]:
reviewcleansed_dataframe.shape

(37352, 17)

In [8]:
##creating positve and negative dataframe
VN= reviewcleansed_dataframe[reviewcleansed_dataframe['bin_word'] =='VN']
QN=reviewcleansed_dataframe[reviewcleansed_dataframe['bin_word'] =='QN']
N= reviewcleansed_dataframe[reviewcleansed_dataframe['bin_word'] =='N']
P=reviewcleansed_dataframe[reviewcleansed_dataframe['bin_word'] =='P']
QP= reviewcleansed_dataframe[reviewcleansed_dataframe['bin_word'] =='QP']
VP=reviewcleansed_dataframe[reviewcleansed_dataframe['bin_word'] =='VP']

In [9]:
VN.shape   #very negative percentage  0.58%

(219, 17)

In [10]:
QN.shape   #quite negative percentage  0.70%

(262, 17)

In [11]:
N.shape    #negative percentage    0.92%

(344, 17)

In [12]:
P.shape   #postive percentage   2.77%

(1035, 17)

In [13]:
QP.shape  #quite positve percentage 20.56%

(7681, 17)

In [14]:
VP.shape    #very positve percentage  74.45%

(27811, 17)

In [15]:
##due to the imbalace in dataset percentage, the team decided to resampling the majority class to follow the negative percentage ratio and size
##resample 219 from very posititve
##resample 262 from quite positive
##resample 344 from positive
##to ensure fairness
##Randomly select unique row from dataframe in Pandas will be used
len(VN['listing_id'].unique().tolist()) ##finding how many unique id there are 177 average comment 1.23
len(QN['listing_id'].unique().tolist()) ##finding how many unique id there are 212 averge comment 1.23
len(N['listing_id'].unique().tolist()) ##finding how many unique id there are 261  averagecomment 1.31 
len(P['listing_id'].unique().tolist()) ##finding how many unique id there are 539  average comment 1.92 
len(QP['listing_id'].unique().tolist()) ##finding how many unique id there are 1274 average comment 6.02 
len(VP['listing_id'].unique().tolist()) ##finding how many unique id there are 1705 average comment 16.3 
#to keep the dataset larage the team decided to random select one comment from each unique id for P,QP,VP based on the
#shape of the VN,QN,N e.g for negative have 344 comments, we will pick a total of 344 comment from the unique ID

1705

## Undersampling method 

In [16]:
#undersampling method
#positive 
P = P.sample(frac=1).drop_duplicates(subset='listing_id')
P = P.sample(n=344)
##quite postive
QP = QP.sample(frac=1).drop_duplicates(subset='listing_id')
QP = QP.sample(n=262)
##very postive
VP = VP.sample(frac=1).drop_duplicates(subset='listing_id')
VP = VP.sample(n=219)
##row id for cleansing of list to syn
VNd = VN
QNd = QN
Nd  = N
Pd  = P
QPd = QP
VPd = VP

In [17]:
pdList = [VNd,QNd,Nd,Pd,QPd,VPd] 
undersample_df = pd.concat(pdList)

In [18]:
undersample_df.shape

(1650, 17)

In [19]:
len(undersample_df['listing_id'].unique().tolist())

921

In [20]:
undersample_df.to_csv('undersample.csv')

## Oversample method 

In [21]:
##oversample implementation
##creating positve and negative dataframe #825
negativeframe = reviewcleansed_dataframe[(reviewcleansed_dataframe['bin_word'] == "N")|(reviewcleansed_dataframe['bin_word']=="QN")|(reviewcleansed_dataframe['bin_word']=="VN")]
positiveframe = reviewcleansed_dataframe[(reviewcleansed_dataframe['bin_word'] == "P")|(reviewcleansed_dataframe['bin_word']=="QP")|(reviewcleansed_dataframe['bin_word']=="VP")]
negativeframe = negativeframe.sample(n=36527,replace=True)


In [22]:
negativeframe.head

<bound method NDFrame.head of        Unnamed: 0  listing_id           id        date  reviewer_id  \
35678       41340  42417293.0  758621787.0   16/5/2021   38194600.0   
10259       11872   8195397.0  532037628.0   19/9/2019   38866195.0   
24296       28200  22021293.0  351473688.0  23/11/2018  170388044.0   
33730       39108  36920538.0  597206832.0   25/1/2020   86569687.0   
36714       42524  45771807.0  753488269.0   30/4/2021   43043281.0   
...           ...         ...          ...         ...          ...   
12195       14107   9563744.0  188889670.0   31/8/2017  147295656.0   
26045       30149  24170288.0  292800518.0   19/7/2018   58842127.0   
2661         2994   3859180.0  219598702.0  18/12/2017  154884659.0   
3599         4049   4380000.0  573723009.0   7/12/2019  264579045.0   
19647       22882  16522880.0  230798056.0   29/1/2018   69380143.0   

      reviewer_name                                           comments  \
35678            Ck  EASY TO CHECK IN AND C

In [23]:
positiveframe.head

<bound method NDFrame.head of        Unnamed: 0  listing_id            id        date  reviewer_id  \
0               0     50646.0  1.190986e+07   18/4/2014    1356099.0   
1               1     50646.0  1.382395e+07    5/6/2014   15222393.0   
2               2     50646.0  1.511722e+07    2/7/2014    5543172.0   
3               3     50646.0  1.542646e+07    8/7/2014     817532.0   
4               4     50646.0  1.555291e+07   11/7/2014   10942382.0   
...           ...         ...           ...         ...          ...   
37347       43234  53388036.0  5.200000e+17  18/12/2021  304470840.0   
37348       43235  53408171.0  5.210000e+17  19/12/2021  433426203.0   
37349       43236  53614218.0  5.190000e+17  17/12/2021  107628510.0   
37350       43237  53679344.0  5.160000e+17  13/12/2021  340715577.0   
37351       43238  53717759.0  5.140000e+17  10/12/2021  192227309.0   

      reviewer_name                                           comments  \
0             James  A COMFORTA

In [24]:
pdList2 = [positiveframe,negativeframe] 
oversample_df = pd.concat(pdList2)
oversample_df.head
#export cleansed data
oversample_df.to_csv('oversample.csv')

In [25]:
len(oversample_df['listing_id'].unique().tolist()) 

1923

In [26]:
##idlist
temp = oversample_df.drop_duplicates(subset='listing_id')
##
iddataframe = temp['listing_id']
iddataframe.to_csv('iddataframe.csv')

In [27]:
##finding all id that is in review to syn 
listingidremoval = pd.read_csv('5.listings.csv')

In [28]:
matching_id = listingidremoval['id'].isin(temp['listing_id']) 
listingidremoval = listingidremoval.loc[matching_id, :]

In [29]:
listingidremoval #1923
listingidremoval.to_csv('5.listing(alreadyremoveuselessidalr).csv')
